# Day 2: Rock Paper Scissors
https://adventofcode.com/2022/day/2

In [2]:
let lines: Vec<String> = std::fs::read_to_string("input/day02.txt")
    .expect("Could not read file")
    .lines()
    .map(|line| line.to_owned())
    .collect();

In [3]:
#[derive(Debug, PartialEq, Clone, Copy)]
enum Shape {
    Rock,
    Paper,
    Scissors
}

In [4]:
impl Shape {
    fn parse(c: char) -> Self {
         match c {
            'A' => Shape::Rock,
            'B' => Shape::Paper,
            'C' => Shape::Scissors,
            'X' => Shape::Rock,
            'Y' => Shape::Paper,
            'Z' => Shape::Scissors,
            _ => panic!("Could not parse shape")
        }
    }

    fn defeats(self: &Self, other: &Shape) -> bool {
        match self {
            Shape::Rock => *other == Shape::Scissors,
            Shape::Paper => *other == Shape::Rock,
            Shape::Scissors => *other == Shape::Paper
        }
    }
}

## Import itertools

In [5]:
:dep itertools = "0.10.5"

In [6]:
use itertools::Itertools;

## Some helper functions

In [7]:
fn parse_round(line: &str) -> (Shape, Shape) {
    line.split_whitespace()
        .map(|word| word.chars().next().map(Shape::parse).unwrap())
        .collect_tuple().unwrap()
}

In [8]:
let rounds: Vec<(Shape, Shape)> = lines.iter()
    .map(|line| parse_round(line))
    .collect();

In [9]:
fn shape_score(shape: &Shape) -> u32 {
    match shape {
        Shape::Rock => 1,
        Shape::Paper => 2,
        Shape::Scissors => 3
    }
}

In [10]:
fn outcome_score(round: &(Shape, Shape)) -> u32 {
    let (opponentShape, myShape) = round;

    if opponentShape == myShape {
        // draw
        3
    } else if myShape.defeats(&opponentShape) {
        // we win
        6
    } else {
        // we lose
        assert!(opponentShape.defeats(&myShape));
        0
    }
}

In [11]:
fn score(round: &(Shape, Shape)) -> u32 {
    let my_shape = &round.1;
    shape_score(my_shape) + outcome_score(round)
}

## Part 1

### Verify given examples

In [12]:
fn verify_example(example: &str) {
    let round = parse_round(example);
    println!("input: {}, round: {:?}, score: {}", example, round, score(&round));
}

verify_example("A Y");
verify_example("B X");
verify_example("C Z");

## Calculate total score

In [13]:
rounds.iter()
    .map(|round| score(&round))
    .sum::<u32>()

input: A Y, round: (Rock, Paper), score: 8
input: B X, round: (Paper, Rock), score: 1
input: C Z, round: (Scissors, Scissors), score: 6


15523

## Part 2

In [14]:
#[derive(Debug)]
enum RoundOutcome {
    Win,
    Draw,
    Loss
}

impl RoundOutcome {
    fn parse(c: char) -> Self {
        match c {
            'X' => Self::Loss,
            'Y' => Self::Draw,
            'Z' => Self::Win,
            _ => panic!("could not parse outcome: {}", c)
        }
    }
}

In [15]:
fn parse_round_part2(line: &str) -> (Shape, RoundOutcome) {
    let mut it = line.split_whitespace();
    let shape = it.next().unwrap().chars().next().map(Shape::parse).unwrap();
    let outcome = it.next().unwrap().chars().next().map(RoundOutcome::parse).unwrap();
    
    (shape, outcome)
}

In [16]:
fn choose_shape(round: &(Shape, RoundOutcome)) -> Shape {
    let (opponent_shape, outcome) = round;
    let all_shapes = [Shape::Rock, Shape::Paper, Shape::Scissors];
    
    *match outcome {
        RoundOutcome::Draw => opponent_shape,
        RoundOutcome::Win => all_shapes.iter().filter(|s| s.defeats(opponent_shape)).next().unwrap(),
        RoundOutcome::Loss => all_shapes.iter().filter(|s| opponent_shape.defeats(s)).next().unwrap(),
    }
}

### Verify given examples

In [17]:
fn verify_example_part2(example: &str) {
    let round = parse_round_part2(example);
    let opponent_shape = round.0;
    let my_shape = choose_shape(&round);
    println!("input: {}, round: {:?}, chosen shape: {:?}, score: {}", example, &round, &my_shape, score(&(opponent_shape, my_shape)));
}

verify_example_part2("A Y");
verify_example_part2("B X");
verify_example_part2("C Z");

### Calculate score

In [18]:
lines.iter()
    .map(|line| parse_round_part2(line))
    .map(|(opponent_shape, outcome)| (opponent_shape, choose_shape(&(opponent_shape, outcome))))
    .map(|round| score(&round))
    .sum::<u32>()

input: A Y, round: (Rock, Draw), chosen shape: Rock, score: 4
input: B X, round: (Paper, Loss), chosen shape: Rock, score: 1
input: C Z, round: (Scissors, Win), chosen shape: Rock, score: 7


15702